## 1. 概述

### 1.1 什么是检索器

- 检索器（Retriever）是 LangChain 中用于从向量数据库检索相关文档的接口。与直接使用向量数据库的搜索方法相比，检索器提供了更高级的抽象和更灵活的检索策略
- LangChain基于Retriever提高了较为复杂的召回策略,可用于从大量文档中检索 和给定查询相关的文档或信息片段的工具,Retriever不需要存储文档,只需要返回/检索文档即可

### 1.2 检索器 vs 向量数据库直接搜索

- **向量数据库直接搜索**：使用 `similarity_search`、`similarity_search_with_score` 等方法，通过计算查询向量与文档向量的相似度来检索
- **检索器（Retriever）**：提供了更高级的抽象，支持多种检索策略（相似度减速、分数阈值检索、MMR、自查询等），可以更好地集成到 LangChain 的链式工作流中

## 2、检索策略

- Retriever的执行步骤:
    - 步骤1: 将输入查询转换为向量表示
    - 步骤2: 在向量存储中搜索 和查询向量最相似的文档向量(通常使用余弦相似度或欧几里得等度量方式)
    - 步骤3: 返回和查询最相关的`k`个文档,以及对应的相似度得分

### 2.0、数据准备(文档加载、分割、嵌入、存储)

In [64]:
from langchain_community.vectorstores import FAISS, Chroma
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAIEmbeddings
import os
import dotenv
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

# 1.创建文档加载器 并加载文档
loader = TextLoader(file_path="./document/load/09-retriever-doc.txt",encoding="utf-8")
docs = loader.load()

print(f"原始文档数量: {len(docs)}")
print(f"原始文档长度: {len(docs[0].page_content)} 字符")

# 2.创建递归字符文本分割器 并分割文档
# chunk_size: 每个块的最大字符数
# chunk_overlap: 相邻块之间的重叠字符数，用于保持上下文连续性
# separator: 分隔符列表，按优先级排序（从高到低）
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,           # 每个块最大500字符
    chunk_overlap=50,         # 相邻块重叠50字符，保持上下文
    separators=["\n\n", "\n", "。", "，", " "],  # 分隔符优先级：段落 > 行 > 句 > 逗号 > 空格
    add_start_index=True       # 展示每个分块在原文中的起始位置
)

# 分割文档
chunks = splitter.split_documents(docs)

print(f"分割后的文档块数量: {len(chunks)}")
print(f"\n前3个文档块的信息:")
for i, chunk in enumerate(chunks[:3], 1):
    print(f"\n块 {i}:")
    print(f"  长度: {len(chunk.page_content)} 字符")
    print(f"  内容预览: {chunk.page_content[:100]}...")
    print(f"  元数据: {chunk.metadata}")

# 3.创建嵌入模型
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# 4. 将文档块及其向量存储到向量数据库
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="./chroma_embeddings-2"
)

原始文档数量: 1
原始文档长度: 3926 字符
分割后的文档块数量: 10

前3个文档块的信息:

块 1:
  长度: 396 字符
  内容预览: 一、杭州西湖
西湖，又称钱塘湖，位于浙江省杭州市中心，是中国著名的淡水湖与文化景观。西湖历史源远流长，历代文人墨客多有题咏，2011 年被列入世界文化遗产，是中国首个以文化景观名义入选的遗产项目。西湖...
  元数据: {'source': './document/load/09-retriever-doc.txt', 'start_index': 0}

块 2:
  长度: 425 字符
  内容预览: 二、泰山
泰山，又称东岳、岱宗，位于山东省泰安市境内，绵亘于泰安、济南、淄博三市之间，是 “五岳之首” 与中华文明的精神标识。泰山历史可追溯至远古，历代帝王多在此封禅祭祀，秦始皇、汉武帝等均留下足迹，...
  元数据: {'source': './document/load/09-retriever-doc.txt', 'start_index': 392}

块 3:
  长度: 402 字符
  内容预览: 三、万里长城
万里长城，又称长城，横亘于中国北方辽阔大地，西起甘肃嘉峪关，东至河北山海关，是世界上规模最大的古代军事防御工程。长城始建于春秋战国时期，秦统一后连接修缮，明代进行大规模扩建，现存主体多为...
  元数据: {'source': './document/load/09-retriever-doc.txt', 'start_index': 811}


### 2.1、使用默认检索器进行相似性检索

In [65]:
retriever = vector_db.as_retriever(
    search_type="similarity",      # 默认检索器类型
    search_kwargs={"k": 2}         # k=2,返回相似度最高的2个结果,默认为4
)
question = "请介绍下山东的旅游景点"
result_doc = retriever.invoke(question)

print("=" * 20)
print(f"查询问题: {question} ")
print(f"len : {len(result_doc)}")
# 10.得到结果
print(f"查询查询结果: ", end="\n\n")
for i, doc in enumerate(result_doc):
    print(f"result{i+1} => ⭐{doc.page_content}")

查询问题: 请介绍下山东的旅游景点 
len : 2
查询查询结果: 

result1 => ⭐二、泰山
泰山，又称东岳、岱宗，位于山东省泰安市境内，绵亘于泰安、济南、淄博三市之间，是 “五岳之首” 与中华文明的精神标识。泰山历史可追溯至远古，历代帝王多在此封禅祭祀，秦始皇、汉武帝等均留下足迹，1987 年被列为世界文化与自然双重遗产。泰山占地面积 426 平方公里，主峰玉皇顶海拔 1545 米，山体雄伟厚重，有 “天下第一山” 之称。
泰山的景观分为丽、旷、幽、奥、妙、秀六大区域，核心景点沿登山古道分布。山麓的岱庙与北京太和殿、曲阜孔庙大成殿并称 “中国古代三大宫殿式建筑”；登山途中的十八盘以陡峭著称，南天门是登顶的标志性节点；山顶天街、碧霞祠等建筑与云海日出景观相映成趣。
泰山拥有 1800 余处历代石刻，被誉为 “中国书法博物馆”，其地质演化历史达 30 亿年，是研究华北地台基底构造的天然标本。作为国家 AAAAA 级景区，泰山每年接待游客超千万人次，承载着 “天人合一” 的文化理念与 “重于泰山” 的精神象征。
三、万里长城
result2 => ⭐五、黄山
黄山，又称黟山，位于安徽省黄山市境内，是中国著名的山岳风景区，以 “奇松、怪石、云海、温泉、冬雪” 五绝著称于世。黄山开发始于唐代，明代旅行家徐霞客两度登临考察，1990 年被列为世界文化与自然双重遗产。景区总面积约 160.6 平方公里，主峰莲花峰海拔 1864.8 米，山体由花岗岩构成，地貌奇特。
黄山的核心景观分布于北海、西海、天都峰、玉屏楼等区域。标志性景观有迎客松、飞来石、光明顶、天都峰等，其中迎客松已成为中国人民热情好客的象征。黄山云海磅礴壮阔，日出时分霞光万道，冬季雾凇与雪景交织，宛如琉璃世界。
黄山植被覆盖率极高，拥有高等植物 1700 余种，野生动物 200 余种，是华东地区的植物宝库与生态屏障。作为国家 AAAAA 级景区，黄山每年接待游客超 600 万人次，其景观成为中国山水画的经典题材，影响深远。
六、苏州古典园林


- ⚠️ FAISS 安装问题解决方案
ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).
如上,遇到 `ImportError: Could not import faiss python package` 错误，请按以下步骤解决：

- 方法1：安装 faiss-cpu（推荐，适用于大多数情况）

```bash
pip install faiss-cpu
```

- 方法2：如果需要 GPU 加速（需要 CUDA 支持）

```bash
pip install faiss-gpu
```

**注意**：faiss-gpu 需要：
- NVIDIA GPU
- CUDA 工具包
- 兼容的 CUDA 版本

### 方法3：指定版本安装（如果默认版本不兼容）

```bash
pip install faiss-cpu==1.7.4
```

### 方法4：使用替代向量数据库（如果 FAISS 安装困难）

如果 FAISS 安装一直有问题，可以考虑使用其他向量数据库：

1. **Chroma**（推荐，易于安装）：
   ```bash
   pip install chromadb
   ```
   使用方式：
   ```python
   from langchain_community.vectorstores import Chroma
   vector_db = Chroma.from_documents(chunks, embedding_model)
   ```

2. **Qdrant**：
   ```bash
   pip install qdrant-client
   ```

3. **Pinecone**（云服务）：
   ```bash
   pip install pinecone-client
   ```

- 重要提示

1. **安装后必须重启 Jupyter 内核**：安装完成后，在 Jupyter 中点击 "Kernel" -> "Restart Kernel"
2. **检查 Python 版本**：确保 Python 版本兼容（通常 Python 3.8+ 都可以）
3. **虚拟环境**：如果使用虚拟环境，确保在正确的环境中安装


### 2.2、分数阈值查询

In [74]:
query = "北京的旅游景点有哪些"

# 推荐先用 similarity_search_with_relevance_scores 检查阈值设置
docs_with_scores = vector_db.similarity_search_with_relevance_scores(query, k=5)
# 过滤掉低于阈值的文档，确保只返回和问题高度相关的内容
score_threshold = 0.12
score_threshold_docs = [
    doc for doc, score in docs_with_scores if score >= score_threshold
]

print(f"len: {len(score_threshold_docs)}")
for i, doc in enumerate(score_threshold_docs):
    print(f"📌 =》内容: {doc.page_content}")

len: 5
📌 =》内容: 北京是中国的首都，拥有丰富的历史文化遗产和现代建筑。以下是一些著名的建筑：

1. 故宫
故宫，又称紫禁城，是明清两代的皇家宫殿，位于北京市中心。它是世界上现存规模最大、保存最完整的木质结构古建筑群之一，1987年被列为世界文化遗产。

2. 天安门
天安门位于故宫南端，是北京的标志性建筑之一。天安门广场是世界上最大的城市广场，可容纳100万人集会。

3. 颐和园
颐和园是清朝时期的皇家园林，以昆明湖和万寿山为基础，融合了江南园林的设计风格，1998年被列入世界文化遗产。
📌 =》内容: 五、黄山
黄山，又称黟山，位于安徽省黄山市境内，是中国著名的山岳风景区，以 “奇松、怪石、云海、温泉、冬雪” 五绝著称于世。黄山开发始于唐代，明代旅行家徐霞客两度登临考察，1990 年被列为世界文化与自然双重遗产。景区总面积约 160.6 平方公里，主峰莲花峰海拔 1864.8 米，山体由花岗岩构成，地貌奇特。
黄山的核心景观分布于北海、西海、天都峰、玉屏楼等区域。标志性景观有迎客松、飞来石、光明顶、天都峰等，其中迎客松已成为中国人民热情好客的象征。黄山云海磅礴壮阔，日出时分霞光万道，冬季雾凇与雪景交织，宛如琉璃世界。
黄山植被覆盖率极高，拥有高等植物 1700 余种，野生动物 200 余种，是华东地区的植物宝库与生态屏障。作为国家 AAAAA 级景区，黄山每年接待游客超 600 万人次，其景观成为中国山水画的经典题材，影响深远。
六、苏州古典园林
📌 =》内容: 五、泰山

泰山位于山东省泰安市，是中国五岳之首，也是世界自然与文化双重遗产。泰山海拔1545米，虽然不是中国最高的山，但因其深厚的历史文化内涵而享有"五岳独尊"的美誉。泰山自古以来就是帝王封禅祭天的圣地，也是文人墨客登临吟咏的胜地。

泰山的自然景观非常壮丽，包括日出、云海、奇松、怪石等自然奇观。泰山的人文景观也非常丰富，包括众多的古建筑、石刻、碑文等。泰山上的石刻数量众多，内容涉及历史、文学、书法等多个方面，具有极高的文化价值。

泰山不仅是自然景观和人文景观的完美结合，更是中华民族精神的象征。登泰山而小天下，体现了中华民族勇于攀登、追求卓越的精神品质。泰山现已成为中国最重要的旅游景点之一，每年吸引着数百万游客前来登山游览。

六、西湖

西湖位于浙江省杭州市，是中国最著名的风景名胜区

- 如上,再基于RAG调用大模型给出回答:

In [72]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")
# 定义提示词模版
template = """
你是一个问答机器人。
你的任务是根据下述给定的已知信息回答用户问题。
确保你的回复完全依据下述已知信息。不要编造答案。
如果下述已知信息不足以回答用户的问题，请直接回复"我无法回答您的问题"。
已知信息:
{context}
用户问：
{question}
请用中文回答用户问题。
"""

prompt_template = PromptTemplate.from_template(template=template)

# question = "北京有哪些杰出的建筑"  # 无法回答问题 => 因为从知识库中检索到文档没有北京的建筑
question = "请介绍下山东省的国家 AAAAA 级景区"
prompt = prompt_template.invoke({"question": question,"context":result_doc})

print("=" * 20)
print(f"result_doc: {result_doc}")
print("=" * 20)
# 将用户提问和相关文档一起交给大模型
response = llm.invoke(prompt)
print(f"content: {response.content}")

result_doc: [Document(metadata={'start_index': 392, 'source': './document/load/09-retriever-doc.txt'}, page_content='二、泰山\n泰山，又称东岳、岱宗，位于山东省泰安市境内，绵亘于泰安、济南、淄博三市之间，是 “五岳之首” 与中华文明的精神标识。泰山历史可追溯至远古，历代帝王多在此封禅祭祀，秦始皇、汉武帝等均留下足迹，1987 年被列为世界文化与自然双重遗产。泰山占地面积 426 平方公里，主峰玉皇顶海拔 1545 米，山体雄伟厚重，有 “天下第一山” 之称。\n泰山的景观分为丽、旷、幽、奥、妙、秀六大区域，核心景点沿登山古道分布。山麓的岱庙与北京太和殿、曲阜孔庙大成殿并称 “中国古代三大宫殿式建筑”；登山途中的十八盘以陡峭著称，南天门是登顶的标志性节点；山顶天街、碧霞祠等建筑与云海日出景观相映成趣。\n泰山拥有 1800 余处历代石刻，被誉为 “中国书法博物馆”，其地质演化历史达 30 亿年，是研究华北地台基底构造的天然标本。作为国家 AAAAA 级景区，泰山每年接待游客超千万人次，承载着 “天人合一” 的文化理念与 “重于泰山” 的精神象征。\n三、万里长城'), Document(metadata={'start_index': 1583, 'source': './document/load/09-retriever-doc.txt'}, page_content='五、黄山\n黄山，又称黟山，位于安徽省黄山市境内，是中国著名的山岳风景区，以 “奇松、怪石、云海、温泉、冬雪” 五绝著称于世。黄山开发始于唐代，明代旅行家徐霞客两度登临考察，1990 年被列为世界文化与自然双重遗产。景区总面积约 160.6 平方公里，主峰莲花峰海拔 1864.8 米，山体由花岗岩构成，地貌奇特。\n黄山的核心景观分布于北海、西海、天都峰、玉屏楼等区域。标志性景观有迎客松、飞来石、光明顶、天都峰等，其中迎客松已成为中国人民热情好客的象征。黄山云海磅礴壮阔，日出时分霞光万道，冬季雾凇与雪景交织，宛如琉璃世界。\n黄山植被覆盖率极高，拥有高等植物 1700 余种，野生动物 200 余种，是华东地区的植物宝库与生态屏障。作为国家 AAAAA 级景区，黄山每年接待游客超 6

- 注意只会返回满足阈值分数的文档，不会获取文档的得分。如果想查询文档的得分是否满足阈值，可以使用向量数据库的

In [73]:
# 使用余弦相似度 =》分数值越接近1（上限），检索到的文档越和问题相似
docs_with_scores = vector_db.similarity_search_with_relevance_scores("请介绍下故宫", 5)

# 根据分数从高到低排序
docs_with_scores_sorted = sorted(docs_with_scores, key=lambda x: x[1], reverse=True)

for doc, score in docs_with_scores_sorted:
    print("=" * 20)
    print(f"\n相似度分数: {score:.10f}")
    print(f"content: {doc.page_content}")


相似度分数: 0.2569482791
content: 二、故宫博物院

故宫，又称紫禁城，位于北京市中心，是明清两朝的皇家宫殿。故宫始建于明成祖永乐年间，历经600多年的历史，是中国现存最大、最完整的古代宫殿建筑群。故宫占地面积约72万平方米，拥有宫殿建筑9000多间，是中国古代宫廷建筑的杰出代表。

故宫的建筑布局严格按照中国古代的礼制思想设计，分为外朝和内廷两大部分。外朝是皇帝处理政务的地方，主要包括太和殿、中和殿、保和殿三大殿。内廷是皇帝和后宫居住的地方，包括乾清宫、交泰殿、坤宁宫等建筑。

故宫内收藏了大量的珍贵文物，包括书画、陶瓷、玉器、青铜器等各类艺术品。这些文物不仅具有极高的艺术价值，更是研究中国古代历史和文化的重要资料。故宫博物院现已成为世界著名的博物馆，每年接待游客超过千万人次。

三、秦始皇兵马俑

秦始皇兵马俑位于陕西省西安市临潼区，是秦始皇陵的陪葬坑。兵马俑发现于1974年，被誉为"世界第八大奇迹"。兵马俑坑共有三个，分别编号为一号坑、二号坑和三号坑，总面积超过两万平方米。

相似度分数: 0.2569482791
content: 二、故宫博物院

故宫，又称紫禁城，位于北京市中心，是明清两朝的皇家宫殿。故宫始建于明成祖永乐年间，历经600多年的历史，是中国现存最大、最完整的古代宫殿建筑群。故宫占地面积约72万平方米，拥有宫殿建筑9000多间，是中国古代宫廷建筑的杰出代表。

故宫的建筑布局严格按照中国古代的礼制思想设计，分为外朝和内廷两大部分。外朝是皇帝处理政务的地方，主要包括太和殿、中和殿、保和殿三大殿。内廷是皇帝和后宫居住的地方，包括乾清宫、交泰殿、坤宁宫等建筑。

故宫内收藏了大量的珍贵文物，包括书画、陶瓷、玉器、青铜器等各类艺术品。这些文物不仅具有极高的艺术价值，更是研究中国古代历史和文化的重要资料。故宫博物院现已成为世界著名的博物馆，每年接待游客超过千万人次。

三、秦始皇兵马俑

秦始皇兵马俑位于陕西省西安市临潼区，是秦始皇陵的陪葬坑。兵马俑发现于1974年，被誉为"世界第八大奇迹"。兵马俑坑共有三个，分别编号为一号坑、二号坑和三号坑，总面积超过两万平方米。

相似度分数: 0.2569482791
content: 二、故宫博物院

故宫，又称紫禁城，位于北京市中心，是明清两朝的皇家宫殿。故宫始建于明成祖永乐年间，历经

### 2.3、MMR检索

#### 1）MMR算法的工作流程

- 步骤1: 向量数据库检索
    - 从所有文档中找出与查询最相似的`fetch_k`个文档
- 步骤2: MMR多样性筛选
    - 从`fetch_k`个候选文档中,使用MMR算法选择`k`个文档
    - 可通过设置`lambda_mult`参数来平衡多样性和相关性,其默认值为0.5,为0则只考虑相关性(类似相似度搜索),为1则只考虑多样性
- 步骤3: 返回结果
    - 返回`k`个既相关又多样化的文档

#### 2）相关参数说明
- 为什么需要 fetch_k？
    - **更大的候选池**：`fetch_k` 越大，MMR 算法有更多选择，能更好地平衡相关性和多样性
    - **避免信息冗余**：如果只返回最相似的文档，可能会返回内容高度重复的文档块
    - **提高结果质量**：通过多样性筛选，确保返回的文档覆盖不同的角度和信息

- 3）参数设置建议
    - **`fetch_k`**：通常设置为 `k` 的 3-10 倍（如 k=5，fetch_k=20-50）
    - **`k`**：根据实际需求设置，通常 3-10 个文档块足够
    - **`lambda_mult`**：根据场景调整
        - 需要高相关性：0.2-0.4
        - 需要高多样性：0.6-0.8

In [52]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document

splitter = CharacterTextSplitter(
    chunk_size=200,           # 每个块最大200字符
    chunk_overlap=20,         # 相邻块重叠20字符，保持上下文
)
docs = [
    Document(page_content="经济复苏：美国经济正在从疫情中强劲复苏，失业率降至历史低点。！"),
    Document(page_content="基础设施：政府将投资1万亿美元用于修复道路、桥梁和宽带网络。"),
    Document(page_content="气候变化：承诺到2030年将温室气体排放量减少50%。"),
    Document(page_content=" 医疗保健：降低处方药价格，扩大医疗保险覆盖范围。"),
    Document(page_content="教育：提供免费的社区大学教育。。"),
    Document(page_content="科技：增加对半导体产业的投资以减少对外国供应链的依赖。。"),
    Document(page_content="外交政策：继续支持乌克兰对抗俄罗斯的侵略。"),
    Document(page_content="枪支管制：呼吁国会通过更严格的枪支管制法律。"),
    Document(page_content="移民改革：提出全面的移民改革方案。"),
    Document(page_content="社会正义：承诺解决系统性种族歧视问题。"),
]

# 分割文档
chunks = splitter.split_documents(docs)

print(f"分割后的文档块数量: {len(chunks)}")

# 4. 将文档块及其向量存储到向量数据库
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="./chroma_embeddings-2" # 指定持久化目录
)
retriever = vector_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 2, "fetch_k": 6}
)
mmr_query_result = retriever.invoke("科技政策")
for doc in mmr_query_result:
    print("内容:", doc.page_content)

分割后的文档块数量: 10
内容: 科技：增加对半导体产业的投资以减少对外国供应链的依赖。。
内容: 医疗保健：降低处方药价格，扩大医疗保险覆盖范围。


- 在基于RAG调用大模型给出回答:

In [59]:
prompt_template = """请使用以下提供的文本内容来回答问题。仅使用提供的文本信息，如果文本中
没有相关信息，请回答"抱歉，提供的文本中没有这个信息"。
文本内容：
{context}
问题：{question}
回答：
"
"""

prompt = PromptTemplate.from_template(template=prompt_template)
# 创建Runnable链
chain = prompt | llm

# question = "请推荐些好书"   # 问个知识库中没有答案的问题
question = "为什么加大对半导体产业的投资"
mmr_rag_result = chain.invoke({"question": question, "context":mmr_query_result})
print(f"content: {mmr_rag_result.content}")

print(f"基于知识库检索得到的文档:",end="\n")
for doc in mmr_query_result:
    print("内容:", doc.page_content)

content: 加大对半导体产业的投资是为了减少对外国供应链的依赖。
基于知识库检索得到的文档:
内容: 科技：增加对半导体产业的投资以减少对外国供应链的依赖。。
内容: 医疗保健：降低处方药价格，扩大医疗保险覆盖范围。


## 3. 检索器的实际应用场景

e### 3.1 RAG 系统中的检索器

在 RAG（Retrieval-Augmented Generation）系统中，检索器的作用是:
- 1）接收用户查询
- 2）将查询转换为向量
- 3）在向量数据库中检索相关文档块
- 4）将检索到的文档块作为上下文传递给 LLM
- 5）LLM 基于检索到的上下文生成回答

### 3.2 检索策略选择

- 1）**相似度搜索**：适用于需要最相关文档的场景，简单直接
- 2）**MMR 搜索**：适用于需要多样化结果的场景，避免信息冗余
- 3）**自查询检索**：适用于需要根据元数据过滤的场景

### 3.3 最佳实践

- 1）**合理设置 chunk_size**: 根据 LLM 和嵌入模型的限制，以及文档特点设置
- 2）**使用 chunk_overlap**: 通常设置为 chunk_size 的 10-20%
- 3）**选择合适的分隔符**: 根据文档结构选择分隔符优先级
- 4）**调整检索参数**: 根据实际效果调整 k 值和 lambda_mult 参数
- 5）**评估检索质量**: 通过相似度分数和实际效果评估检索质量